# Huggging Face with langchain with OpenSource Models

In [24]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

from dotenv import load_dotenv

load_dotenv()
import os
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")


## Read the PDFs from the folder

In [2]:
loader = PyPDFDirectoryLoader("./us-census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us-census\\acsbr-015.pdf', 'page': 0})

In [3]:
print("Total number of final documents: ", len(final_documents))

Total number of final documents:  316


## Embeddings Techniques from HugginFace

In [4]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
        model_kwargs={'device' : 'cpu'}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\Documents\tutorials\Langchain Course Krishanik\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amuly\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import numpy as np

embeddings = np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))
embeddings

array([-8.46568272e-02, -1.19099692e-02, -3.37892324e-02,  2.94558685e-02,
        5.19160032e-02,  5.73839732e-02, -4.10018303e-02,  2.74267700e-02,
       -1.05128214e-01, -1.58055797e-02,  7.94858411e-02,  5.64318299e-02,
       -1.31765343e-02, -3.41544412e-02,  5.81596605e-03,  4.72547710e-02,
       -1.30746840e-02,  3.12989787e-03, -3.44225727e-02,  3.08406614e-02,
       -4.09086421e-02,  3.52737978e-02, -2.43761782e-02, -4.35830951e-02,
        2.41503250e-02,  1.31986942e-02, -4.84450767e-03,  1.92346983e-02,
       -5.43912984e-02, -1.42735004e-01,  5.15532214e-03,  2.93115582e-02,
       -5.60810305e-02, -8.53529107e-03,  3.14141288e-02,  2.76736263e-02,
       -2.06188112e-02,  8.24231654e-02,  4.15425114e-02,  5.79654388e-02,
       -3.71587053e-02,  6.26160344e-03, -2.41389666e-02, -5.61805395e-03,
       -2.51715202e-02,  5.04971109e-03, -2.52801627e-02, -2.91945296e-03,
       -8.24043062e-03, -5.69604039e-02,  2.30822973e-02, -5.54217212e-03,
        5.11555262e-02,  

In [6]:
# View the shape of the embeddings array
print(f"Shape of the Embeddings array: {embeddings.shape}")

Shape of the Embeddings array: (384,)


## Vector Store Creation

In [7]:
## Vector Store Creation
vectorstore = FAISS.from_documents(final_documents[:120], huggingface_embeddings)

In [8]:
# Query using Similarity Search
query = "WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_documents = vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


## create Retriever 

In [25]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k" : 3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000248078AF370> search_kwargs={'k': 3}


## Create the model

In [21]:
from langchain_community.llms import HuggingFaceEndpoint
hf=HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.5,
    token=HF_TOKEN

)
query="What is the health insurance coverage?"
hf.invoke(query)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\amuly\.cache\huggingface\token
Login successful


"\n\nThe health insurance coverage is a benefit provided by the employer to the employees. It covers the medical expenses incurred by the employees and their dependents. The coverage includes hospitalization, surgery, consultation fees, and other medical expenses.\n\nWhat is the difference between individual and group health insurance?\n\nIndividual health insurance is a policy purchased by an individual for themselves, while group health insurance is a policy purchased by an employer for their employees. The premiums for group health insurance are generally lower than individual health insurance due to the larger pool of insured individuals.\n\nWhat is the difference between an HMO and a PPO?\n\nAn HMO (Health Maintenance Organization) is a type of health insurance plan that requires the insured to use doctors and hospitals within the HMO network for most services. A PPO (Preferred Provider Organization) is a type of health insurance plan that allows the insured to use doctors and hos

Hugging Face models can be run locally through the HuggingFacePipeline class


In [17]:
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# 
# hf = HuggingFacePipeline.from_model_id(
#         model_id = "mistralai/Mistral-7B-v0.1",
#         task="text-generation",
#         pipeline_kwargs={'temperature' : 0, "max_new_tokens": 300}
# )
# 
# # Testing the model
# query="What is the health insurance coverage?"
# hf.invoke(query)

In [22]:
prompt_template = """
Use the following piece of context to answer the question asked. Please try to provide the answer only based on the context

{context}
Question: {question}

Helpful Answers:
"""

In [23]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [26]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [29]:
query= """DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [30]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


1. In 2022, the uninsured rate ranged from a low of 2.4% in Massachusetts to a high of 16.6% in Texas, with a national rate of 8.0%.
2. Ten of the 15 states with the highest uninsured rates had rates above the national average.
3. Medicaid coverage accounted for a larger percentage of the population in states that expanded Medicaid eligibility compared to nonexpansion states (22.7% vs. 18.0%).
4. Uninsured rates decreased in 27 states from 2021 to 2022, while only Maine had an increase.
5. The private coverage rates were not statistically different in North Dakota and Utah in 2022.

Explanation:

The context discusses the differences in the uninsured rate by state in 2022, with a focus on private coverage rates and Medicaid coverage. It also mentions changes in the uninsured rate from 2021 to 2022. The text provides data on the percentage of uninsured people in various states and metropolitan areas, as well as a comparison of ACS and CPS ASEC measures of health insurance coverage.

Th